#### Note:

Make sure that you have these two scripts running at the same time while running this

ark regsitry start

python sim_node.py

In [588]:
import time
import numpy as np
from arktypes import joint_group_command_t, task_space_command_t, joint_state_t
from arktypes.utils import pack, unpack
from ark.client.comm_infrastructure.instance_node import InstanceNode

ROBOT_NAME = "arkbot"
SIM = False

class HarkControllerNode(InstanceNode):
    def __init__(self):
        super().__init__("arkbot_controller")
        suffix = "/sim" if SIM else ""
        base = f"{ROBOT_NAME}"

        # pubs
        self.joint_group_command = self.create_publisher(f"{base}/joint_group_command{suffix}", joint_group_command_t)
        self.task_space_command  = self.create_publisher(f"{base}/cartesian_command{suffix}", task_space_command_t)

        # subs
        self.state = self.create_listener(f"{base}/joint_states{suffix}", joint_state_t)

node = HarkControllerNode()

[WARNING] [10:58:21.309584] - No global configuration provided. Using default system configuration.
[INFO] [10:58:21.373143] - Service: Successfully registered '__DEFAULT_SERVICE/GetInfo/arkbot_controller_9df39025-331c-4d31-98ee-75a0c2423d51' with registry.
[INFO] [10:58:21.376438] - Service: Successfully registered '__DEFAULT_SERVICE/SuspendNode/arkbot_controller_9df39025-331c-4d31-98ee-75a0c2423d51' with registry.
[INFO] [10:58:21.380106] - Service: Successfully registered '__DEFAULT_SERVICE/RestartNode/arkbot_controller_9df39025-331c-4d31-98ee-75a0c2423d51' with registry.
[ERROR] [10:58:21.381417] - Couldn't load config for other 'arkbot_controller'
[OK] [10:58:21.382499] - setup publisher arkbot/joint_group_command[joint_group_command_t]
[OK] [10:58:21.383105] - setup publisher arkbot/cartesian_command[task_space_command_t]
[OK] [10:58:21.383858] - subscribed to arkbot/joint_states[joint_state_t]


None


---

# Joint Group Control

In [288]:
all_cmd = [
    0.0,   # Revolute 1 (continuous)
     0.0,   # Revolute 2 (rad)
     0.0,   # Revolute 4 (rad)
     0.00,   # Revolute 5 (continuous)
     0.00,   # Revolute 6 (rad)
     0.000   # Slider 7 position in meters (range 0.0 .. 0.0425)
]
node.joint_group_command.publish(pack.joint_group_command(all_cmd, "all"))

In [577]:
home =  [0.0, 0.0, 0.0, 0.00, 0.0, 0.0, 0.0]
approach_upper = [0.4, 0, -1.2, 2.0, 0.0, 2.2, 0.0]
contact_upper = [0.4, 0.3, -1.2, 2.0, 0.0, 2.2, 0.0]
push_upper = [0.4, 0.3, -1.2, 1.35, 0.0, 1.4, 0.0]
lower_upper = [0.4, 0.3, -1.2, 2.0, 0.0, 2.2, 0.0]
approach_lower = [0.4, 0.3, -1.8, 2.0, 0.0, 2.2, 0.0]
contact_lower = [0.4, 0.4, -1.8, 1.5, 0.0, 2.2, 0.0]
push_lower = [0.4, 0.4, -1.8, 0.6, 0.0, 1.5, 0.0]
retract_lower = [0.4, -0.3, -1.8, 2.0, 0.0, 2.2, 0.0]


In [583]:
node.joint_group_command.publish(pack.joint_group_command(home, "all"))

In [584]:
node.joint_group_command.publish(pack.joint_group_command(approach_upper, "all"))

In [580]:
node.joint_group_command.publish(pack.joint_group_command(contact_upper, "all"))

In [565]:
node.joint_group_command.publish(pack.joint_group_command(push_upper, "all"))

In [567]:
node.joint_group_command.publish(pack.joint_group_command(lower_upper, "all"))

In [568]:
node.joint_group_command.publish(pack.joint_group_command(approach_lower, "all"))

In [569]:
node.joint_group_command.publish(pack.joint_group_command(contact_lower, "all"))

In [573]:
node.joint_group_command.publish(pack.joint_group_command(push_lower, "all"))

In [574]:
node.joint_group_command.publish(pack.joint_group_command(retract_lower, "all"))

In [554]:
node.joint_group_command.publish(pack.joint_group_command(home, "all"))

In [ ]:
# save the position offset, click this once when the robot is in the zero position
# unpack.joint_state(node.state.get())

joint_state_data = unpack.joint_state(node.state.get())

joint_positions = joint_state_data[2]

print(joint_positions)



[ 0.        -2.7925268 -3.4906585  0.         0.         0.
  0.       ]


In [7]:
# GRIPPER (velocity mode): single value for Slider 7 velocity (m/s)
gripper_vel_open  = [ +0.010 ]   # open slowly
gripper_vel_close = [ -0.010 ]   # close slowly
node.joint_group_command.publish(pack.joint_group_command(gripper_vel_open, "gripper"))

In [8]:
# ARM (position mode): 6 values, Revolute 1..6
arm_cmd = [-0.30, 0.50, 0.30, -1.00, 0.10, 0.80]
node.joint_group_command.publish(pack.joint_group_command(arm_cmd, "arm"))


---

# Task Space Control

In [7]:
xyz_command = np.array([0.3, 0.4, 0.8])
quaternion_command = np.array([1, 0.0, 0.0, 0.0]) # xyz-w
gripper = 1.0 # 0.0 close, 1.0 open

controller.task_space_command.publish(
    pack.task_space_command("all", xyz_command, quaternion_command, gripper))

---


# Joint States

In [287]:
unpack.joint_state(node.state.get())


({'seq': 0, 'stamp': {'sec': 0, 'nsec': 0}, 'frame_id': ''},
 ['Revolute 1',
  'Revolute 2',
  'Revolute 3',
  'Revolute 4',
  'Revolute 5',
  'Revolute 6',
  'Slider 7'],
 array([0.99998503, 0.99998503, 0.        , 0.        , 0.        ,
        0.        , 0.        ]),
 array([0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0.]))